In [6]:
import sys
sys.path.append("../")
from broncode.apple_classifyer import AppleClassifyer

from sentence_transformers import SentenceTransformer, util
import torchvision.transforms as T
import torch.nn.functional
from torchvision.io import read_image, ImageReadMode
import torch
import pickle as pkl
import os

In [3]:
print(os.getcwd())

c:\Users\nilsm\workspace\MakeAIWork3\Project 3\notebooks


In [4]:
apple_model_path = "../models/AppleClassifyer_86"

In [7]:
load = open(apple_model_path, "rb")
apple_classifyer = pkl.load(load)

In [11]:
aql_sample20 = []
aql_path = "../informatie/apple_disease_classification/images/Test/AQL_testbatch/"

for filename in os.listdir(aql_path):
    aql_sample20.append(read_image(aql_path+filename, ImageReadMode.RGB)/255)

print(aql_sample20)

[tensor([[[0.4471, 0.4627, 0.4784,  ..., 0.3961, 0.3725, 0.3569],
         [0.4510, 0.4588, 0.4706,  ..., 0.4118, 0.3804, 0.3608],
         [0.4549, 0.4588, 0.4667,  ..., 0.4196, 0.3882, 0.3647],
         ...,
         [0.4784, 0.4431, 0.3922,  ..., 0.3843, 0.3804, 0.3765],
         [0.4471, 0.4157, 0.3765,  ..., 0.3725, 0.3804, 0.3843],
         [0.4235, 0.3961, 0.3686,  ..., 0.3608, 0.3765, 0.3882]],

        [[0.4196, 0.4353, 0.4510,  ..., 0.4078, 0.3843, 0.3686],
         [0.4235, 0.4314, 0.4431,  ..., 0.4235, 0.3922, 0.3725],
         [0.4275, 0.4314, 0.4392,  ..., 0.4314, 0.4000, 0.3765],
         ...,
         [0.4627, 0.4275, 0.3765,  ..., 0.3922, 0.3882, 0.3843],
         [0.4314, 0.4000, 0.3608,  ..., 0.3804, 0.3882, 0.3922],
         [0.4078, 0.3804, 0.3529,  ..., 0.3686, 0.3843, 0.3961]],

        [[0.3882, 0.4039, 0.4196,  ..., 0.3804, 0.3569, 0.3412],
         [0.3922, 0.4000, 0.4118,  ..., 0.3961, 0.3647, 0.3451],
         [0.3961, 0.4000, 0.4078,  ..., 0.4039, 0.3725, 0

In [12]:
resize = T.Resize((64,64))
print(aql_sample20[0].shape)

for i in range(len(aql_sample20)):
    aql_sample20[i] = resize(aql_sample20[i])
    aql_sample20[i] = aql_sample20[i].unsqueeze(0)

print(aql_sample20[0].shape)

torch.Size([3, 312, 251])
torch.Size([1, 3, 64, 64])


In [13]:
aql, good_amount, bad_amount = apple_classifyer.aql_classifyer(aql_sample20)

print(f"AQL quality from code letter F: {aql}")

Amount of good apples: 11/20
AQL quality from code letter F: 15


In [15]:
if aql == 0.4:
    apple_use = "Approved for: grocery/greengrocer"
elif aql == 6.5:
    apple_use = "Approved for apple sauce"
elif aql == 15:
    apple_use = "Approved for apple syrup"
elif aql == "REJECTED":
    apple_use = aql


total_apples = good_amount + bad_amount
percentage = round((good_amount*100) / total_apples)

print(apple_use)

Approved for apple syrup


<h1>QUESTIONS</h1>

Ask one of these questions after running the cell below:
-   What is the AQL of these apples?
-   How many good apples are in this sample?
-   How many bad apples are in this sample?
-   How many apples are in this sample?
-   How do you classify your apples?

In [20]:
chat_model = SentenceTransformer('all-MiniLM-L12-v2')

query_embedding = chat_model.encode(input('What is your question?'))

possible_answers = [f"From the {total_apples} apples in this sample {good_amount} are good, so thats {percentage}%.",
                    'How I classify apples is my little secret. >:)',
                    f'There are {(total_apples-good_amount)} bad apples in this sample.',
                    f"There are {total_apples} apples in this sample.",
                    f"The AQL of these apples is {aql}, this means that the apples are: {apple_use}"]

passage_embedding = chat_model.encode(possible_answers)

answer = util.dot_score(query_embedding, passage_embedding)
v, idx = torch.max(answer, 1)
print(possible_answers[idx])

There are 20 apples in this sample.
